# Pull Requests

Pull Requests is a collaborative process of reviewing code changes before they are merged into the main code branch.

Goal of code reviews is to increase code quality.

Pull requests may sometimes be called "Merge requests" or similar names, based on the platform used.

## Pull requests

Pull requests allow to get second (or even more) look on changes before they become they are merged into the main trunk. Pull requests involves other engineers looking through the submited changes and providing comments on what could potentially be improved.

Pull requests (and code reviews) in general are considered one of the *best practices* in software engineering. If done correctly they can help engineering teams to prevent bugs from slipping into production and ensuring that the code will be understood in the future.

### Some tips for pull requests

When reviewing a pull request it is best to focus only on what was changed with changes included in this pull request. Sometimes it so happens that the pull requests highlights issues with code decisions that were made in the past. While it might be tempting to give comments on how these old changes should be reworked to better suit current situation, usually that is counter productive. It will only likely drag on the process of getting changes into production and potentially create disdain for interacting with older code.

Provide suggestions for the problems that you raise. It is not that helpful for other engineer when you review the pull request and just state that "this or that" is wrong and should be made better. It is best if you provide actionable points on how exactly the code could be improved.

Prioritise helping other learn, rather that scold. While there is some inherent friction in the pull request review process, because it is inherently asymmetrical, as a reviewer, try to make it as pleasant as possible for the reviewee. Don't go into blame games or trying to show the "smart way of doing things". Provide actionable solutions and focus on getting the code good enough, rather than perfect. Code usually can be infinitely optimised, but if the current code does what it is supposed to do and is sufficiently maintainable - it is probably good enough.

## Raising PRs

- Provide context what is being changed.
- The smaller the PR - the easier it is to review it.
- 

## Reviewing PRs

- Focus on what is in the scope of the PR.

### Code readability

One of the goals of the pull request is to make sure that the code that was produced is *readable*. By readable it means that other engineer than the author can easily understand what the code is doing. Doing this usually involves making sure that the variables, classes and methods are appropriately and descriptively named.

Make sure that you as well as others will be able to understand the code well enough in the future to maintain it.

A bit more on readable code: [https://blog.pragmaticengineer.com/readable-code/](https://blog.pragmaticengineer.com/readable-code/).

### Example 1: Naming & Intent - Before

What's wrong with this code? What does it even do?

```csharp
class U
{
    public int C(List<int> l)
    {
        var s = 0;
        foreach (var i in l) s += i;
        return s / l.Count;
    }
}
```

### Example 1: Naming & Intent - After

Clearer names, validation, and correct types prevent bugs and improve readability.

```csharp
static class Statistics
{
    public static double Average(IReadOnlyList<int> numbers)
    {
        if (numbers is null) throw new ArgumentNullException(nameof(numbers));
        if (numbers.Count == 0) throw new ArgumentException("Sequence cannot be empty.", nameof(numbers));

        var sum = numbers.Sum();
        return (double)sum / numbers.Count;
    }
}
```

Generally comments can be grouped into 2 categories:
1. Readability related comments
2. Code logic related comments

- https://linearb.io/resources/engineering-benchmarks Some metrics and reference values to think about when reviewing you PR process.


### Example 2: Separation of Concerns - Before

How would you test this? What are the different responsibilities here?

```csharp
class OrderProc
{
    public decimal P(string path)
    {
        var json = File.ReadAllText(path);
        var order = JsonSerializer.Deserialize<Order>(json);
        if (order.DiscountCode == "VIP") return order.Amount * 0.8m;
        return order.Amount;
    }
}

class Order { public decimal Amount { get; set; } public string DiscountCode { get; set; } }
```

### Example 2: Separation of Concerns - After

Separating file access from pricing logic makes the code testable and easier to understand.

```csharp
static class OrderReader
{
    public static Order ReadFromFile(string path) =>
        JsonSerializer.Deserialize<Order>(File.ReadAllText(path))
        ?? throw new InvalidDataException("Order payload invalid");
}

static class Pricing
{
    public static decimal ApplyDiscount(Order order)
    {
        if (order is null) throw new ArgumentNullException(nameof(order));
        return order.DiscountCode switch
        {
            "VIP" => order.Amount * 0.8m,
            "EMP" => order.Amount * 0.7m,
            _ => order.Amount
        };
    }
}
```

### Example 3: Incremental Refactoring - Before

This logic is hard to review. How could you make the change easier to verify in a PR?

```csharp
class ShippingCostCalculatorV1
{
    public decimal Calculate(string country, int weightGrams)
    {
        var baseCost = 5m;
        if (country == "US") baseCost = 3m;
        if (weightGrams > 1000) baseCost += 4m;
        if (country == "DE" && weightGrams < 500) baseCost -= 1m;
        return baseCost;
    }
}
```

### Example 3: Incremental Refactoring - After

Extracting decisions into small, pure functions makes the change easy to review and test.

```csharp
class ShippingCostCalculatorV2
{
    public decimal Calculate(string country, int weightGrams)
        => Base(country) + Surcharge(weightGrams) + CountryAdjustment(country, weightGrams);

    private static decimal Base(string country) => country == "US" ? 3m : 5m;
    private static decimal Surcharge(int weightGrams) => weightGrams > 1000 ? 4m : 0m;
    private static decimal CountryAdjustment(string country, int weightGrams)
        => (country, weightGrams) switch { ("DE", < 500) => -1m, _ => 0m };
}
```

### Example 4: Clear Contracts - Before

What happens on failure? What are the valid inputs for `e` and `p`?

```csharp
class UserServiceV1
{
    public bool Register(string e, string p)
    {
        if (e.Contains("@") && p.Length > 5)
        {
            // pretend save
            return true;
        }
        return false;
    }
}
```

### Example 4: Clear Contracts - After

Exceptions and specific validation create a clear, enforceable contract.

```csharp
class InvalidEmailException : ArgumentException { /* ... */ }

static class Email
{
    static readonly Regex Pattern = new Regex(/* ... */);
    public static void EnsureValid(string value, string paramName)
    {
        if (string.IsNullOrWhiteSpace(value) || !Pattern.IsMatch(value))
            throw new InvalidEmailException("Email format invalid", paramName);
    }
}

class UserServiceV2
{
    public void Register(string email, string password)
    {
        Email.EnsureValid(email, nameof(email));
        if (string.IsNullOrWhiteSpace(password) || password.Length < 6)
            throw new ArgumentException("Password must be at least 6 characters.", nameof(password));
        // persist user here
    }
}
```

### Comments - Why focus on them?

- Comments should explain the why, constraints, and trade-offs — not restate code.
- Next slides: a quick before/after illustrating helpful vs redundant comments.